In [34]:
import numpy as np
import os
import pickle
import librosa
import pandas
from models import cross_validation
from feature import *

%load_ext autoreload
%autoreload 1
%aimport models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:

# -------- Get data --------------
dataset_path = r"dataset/Coswara-Data/preprocess2"
raw_data_path = os.path.join(dataset_path, "vat_48000.pkl")

sample_rate = 48000
frame_length = 2048
hop_length = frame_length // 2

with open(raw_data_path, "rb") as f:
    data = pickle.load(f)

signals = np.asarray(data["signal"])
labels = np.asarray(data["label"])

# --------- Features extraction -----------

mfcc39 = librosa.feature.mfcc(y = signals, n_mfcc = 39, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(mfcc39.shape)
mfcc26 = librosa.feature.mfcc(y = signals, n_mfcc = 26, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(mfcc26.shape)
mfcc13 = librosa.feature.mfcc(y = signals, n_mfcc = 13, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(mfcc13.shape)
chroma = librosa.feature.chroma_stft(y = signals, sr = sample_rate, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(chroma.shape)
mel = librosa.feature.melspectrogram(y = signals, sr = sample_rate, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(mel.shape)
cent = librosa.feature.spectral_centroid(y = signals, sr = sample_rate, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(cent.shape)
bandwidth = librosa.feature.spectral_bandwidth(y = signals, sr = sample_rate, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(bandwidth.shape)
flatness = librosa.feature.spectral_flatness(y = signals, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(flatness.shape)
onset = librosa.onset.onset_strength(y = signals,  sr = sample_rate, n_fft = frame_length, hop_length = hop_length)[:, None, :].astype(np.float32)
print(onset.shape)
cqt = librosa.cqt(y = signals, sr = sample_rate, hop_length = hop_length).astype(np.float32)
print(cqt.shape)
zcr = librosa.feature.zero_crossing_rate(signals, frame_length = frame_length, hop_length = hop_length).astype(np.float32)
print(zcr.shape)
rolloff = librosa.feature.spectral_rolloff(y = signals, sr = sample_rate, n_fft = frame_length, hop_length = hop_length).astype(np.float32)
print(rolloff.shape)
contrast = librosa.feature.spectral_contrast(y = signals, sr = sample_rate, n_fft = frame_length, hop_length = hop_length, n_bands = 5).astype(np.float32)
print(contrast.shape)



(1092, 39, 47)
(1092, 26, 47)
(1092, 13, 47)
(1092, 12, 47)
(1092, 128, 47)
(1092, 1, 47)
(1092, 1, 47)
(1092, 1, 47)
(1092, 1, 47)


/home/xiangyu/anaconda3/envs/torch171/lib/python3.7/site-packages/ipykernel_launcher.py:35: ComplexWarning: Casting complex values to real discards the imaginary part


(1092, 84, 47)
(1092, 1, 47)
(1092, 1, 47)
(1092, 6, 47)


In [39]:
# --------------- Training -----------------
selected = []
for metric in ["losses", "accuracies", "aucs"]:
    
    features = [mfcc39, mfcc26, mfcc13, chroma, mel, cent, bandwidth, flatness, onset, zcr, rolloff, contrast]
    flags = np.zeros(len(features))
    names = np.asarray([vname(feature, globals()) for feature in features])
    losses = np.full((len(features), len(features)), np.inf, dtype = np.float32)
    accuracies = np.full((len(features), len(features)), -1, dtype = np.float32)
    aucs = np.full((len(features), len(features)), -1, dtype = np.float32)

    fs = ForwardSelection(
        features = features,
        labels = labels, 
        flags = flags, 
        names = names,
        losses = losses, 
        accuracies = accuracies, 
        aucs = aucs, 
        out_dir = "cross_val"
        )

    selected.append(fs.forward_selection(metric = metric))

Selected feature: ['mfcc39']
cuda:0
[2022-10-07 04:07:48] [epoch:1, batch:  5] training loss: 0.6975, training accuracy: 0.486, validation loss: 0.6682, validation accuracy: 0.607, time used:  0.2 s, total time used:  0.2 s.
[2022-10-07 04:07:48] [epoch:2, batch:  5] training loss: 0.6355, training accuracy: 0.646, validation loss: 0.6433, validation accuracy: 0.621, time used:  0.6 s, total time used:  0.6 s.
[2022-10-07 04:07:49] [epoch:3, batch:  5] training loss: 0.6159, training accuracy: 0.678, validation loss: 0.6303, validation accuracy: 0.644, time used:  0.9 s, total time used:  0.9 s.
[2022-10-07 04:07:49] [epoch:4, batch:  5] training loss: 0.5701, training accuracy: 0.730, validation loss: 0.6234, validation accuracy: 0.644, time used:  1.1 s, total time used:  1.1 s.
[2022-10-07 04:07:49] [epoch:5, batch:  5] training loss: 0.5006, training accuracy: 0.762, validation loss: 0.6201, validation accuracy: 0.653, time used:  1.4 s, total time used:  1.4 s.
[2022-10-07 04:07:4

In [50]:
pandas.DataFrame(fs.accuracies, columns = names)

,mfcc39,mfcc26,mfcc13,chroma,mel,cent,bandwidth,flatness,onset,zcr,rolloff,contrast
0,0.69961,0.683126,0.683164,0.579691,0.601688,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.643777
1,-1.00000,0.687721,0.687713,0.698685,0.663956,0.697780,0.705107,0.696858,0.693205,0.685870,0.700557,0.687688
2,-1.00000,0.698718,0.704189,0.702358,0.673064,0.701449,-1.000000,0.705132,0.703280,0.702354,0.701437,0.700540
3,-1.00000,0.706041,0.704202,0.694123,0.685891,0.701441,-1.000000,0.699610,0.698718,-1.000000,0.709681,0.692267
4,-1.00000,0.699602,0.706041,0.709698,0.684052,-1.000000,-1.000000,0.701445,0.712421,-1.000000,0.701462,0.702350
5,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
6,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
7,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
8,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
9,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
